# Big Data in Finance: Part II (Lecture 1) <BR><BR> Lecture Notes 4: Merge CRSP and Compustat 
    
**Instructor** <BR>
Lira Mota, lmota20@gsb.columbia.edu
    
**TA's:** <BR>
Pu He, phe19@gsb.columbia.edu <BR> Daheng Yang, dyang22@gsb.columbia.edu

**Objective:** <BR>
    
Create a final data set called "stock_annual" that has both crsp (permno/permco) and compustat (gvkey) identifiers.

## Query CRSP/Compustat merged table (annual or quarterly).

1. Output table is CRSP-centric with permno-datadate being the primary key.
2. Fiscal period end date must be within link date range.
3. WRDS SAS code can be found [here](https://wrds-web.wharton.upenn.edu/wrds/support/Data/_003Sample%20Programs/CRSP/index.cfm?).

## Set Up

In [1]:
#%% Packages 
import wrds
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
from time import strptime, strftime

# Setups
pd.set_option("display.max_rows", 100) # max number or rows to be displayed 
%matplotlib inline 
plt.rcParams['figure.figsize'] = [10, 6]

In [2]:
db = wrds.Connection(wrds_username='lmota')  # make sure to configure wrds connector before hand.

Loading library list...
Done


## Many Linking Tables

In [3]:
crsp_tables = db.list_tables(library="crspq") # compd: Compustat daily update
pd.DataFrame({'tables':crsp_tables}) # Transform libs to a Pandas data frame to have a better display.

,tables
0,acti
1,asia
2,asib
3,asic
4,asio
5,asix
6,bmdebt
7,bmheader
8,bmpaymts
9,bmquotes


**CCMXPF_LNKHIST - Link History**


* CCMXPF_LNKHIST contains the historical and current links between GVKEY and LPERMNO/LPERMCO (previously known as NPERMN O/NPERMCO in old version CCM). 


* In addition, it includes the beginning and ending dates for links, the link types, and the primary issue marker, and liid codes (Compustat's secondary ID variable, IID).

[Source](https://wrds-www.wharton.upenn.edu/pages/support/research-wrds/research-guides/ccm-sas-datasets/)

## Many Linking Types

In [4]:
db.describe_table(library="crspq", table="ccmxpf_lnkhist")

Approximately 101711 rows in crspq.ccmxpf_lnkhist.


,name,nullable,type
0,gvkey,True,VARCHAR(6)
1,linkprim,True,VARCHAR(1)
2,liid,True,VARCHAR(3)
3,linktype,True,VARCHAR(2)
4,lpermno,True,DOUBLE PRECISION
5,lpermco,True,DOUBLE PRECISION
6,linkdt,True,DATE
7,linkenddt,True,DATE


LINKPRIM: Primary issue marker for the link.

## Download Data

In [5]:
sql = '''SELECT gvkey, lpermno as permno, lpermco as permco, linktype, linkdt, linkenddt
         FROM crspq.ccmxpf_lnkhist
         WHERE linktype IN ('LC', 'LU', 'LS') 
      ''' 

link_table = db.raw_sql(sql, date_cols=['linkdt', 'linkenddt'])

In [6]:
compa = pd.read_pickle('C:/Users/lmota20/Dropbox/Columbia/BigData/class2019/Lecture_I/outputs/compa_annual.pkl')

## Understading Linking Table

In [7]:
link_table[link_table.permco==54311]

,gvkey,permno,permco,linktype,linkdt,linkenddt
14499,017010,13757.0,54311.0,LC,2013-01-14,2016-04-17
14500,017010,13758.0,54311.0,LC,2013-01-14,2016-04-15
14501,017010,14811.0,54311.0,LC,2014-07-24,2016-04-17
14502,017010,15998.0,54311.0,LC,2016-04-18,2017-12-31
14503,017010,15998.0,54311.0,LC,2018-01-01,NaT
14504,017010,15999.0,54311.0,LC,2016-04-16,NaT
14505,017010,16000.0,54311.0,LC,2016-04-18,2017-12-31
14506,017010,16000.0,54311.0,LC,2018-01-01,NaT
18949,027186,15996.0,54311.0,LC,2016-04-18,2017-12-31
18950,027186,15996.0,54311.0,LC,2018-01-01,NaT


## Merge and Link

* Link type code. Each link is given a code describing the connection between the CRSP and Compustat data. 

* Values are:

    * LC – Link research complete. Standard connection between databases.
    * LU – Unresearched link to issue by CUSIP
    * LS – Link valid for this security only. Other CRSP PERMNOs with the same PERMCO will link to other GVKEYs.
    * LX – Link to a security that trades on another exchange system not included in CRSP data.
    * LD – Duplicate Link to a security. Another GVKEY/IID is a better link to that CRSP record.
    * LN – Primary link exists but Compustat does not have prices.
    * NR – No link available, confirmed by research
    * NU – No link available, not yet confirmed

In [8]:
link_table.linktype.value_counts()

LU    16277
LC    13355
LS     3030
Name: linktype, dtype: int64

In [9]:
df = pd.merge(compa, link_table, on=['gvkey'], how='inner')

In [10]:
df = df[(df.datadate >= df.linkdt) | (df.linkdt.isnull())]
df = df[(df.datadate <= df.linkenddt) | (df.linkenddt.isnull())]

In [11]:
df['permno'] = df['permno'].astype(int)
df['permco'] = df['permco'].astype(int)
df['datadate'] = pd.to_datetime(df['datadate'])

In [16]:
df.shape

(261633, 53)

In [17]:
compa.shape

(407279, 48)

In [29]:
df.duplicated(subset=['permno', 'datadate']).sum()

0

## Special Cases

### Many PERMCOs for One GVKEY

In [19]:
df.groupby('gvkey').permco.nunique().max()

4

In [20]:
df.loc[df.gvkey==df.groupby('gvkey').permco.nunique().idxmax(), ['datadate', 'permno', 'permco','gvkey', 'conm', 'at']]

,datadate,permno,permco,gvkey,conm,at
291345,1995-12-31,81875,4388,032280,STARZ,2517.636
291352,1996-12-31,81875,4388,032280,STARZ,3058.952
291359,1997-12-31,81875,4388,032280,STARZ,5038.561
291366,1998-12-31,81875,4388,032280,STARZ,20348.000
291374,1999-12-31,86746,20103,032280,STARZ,58684.000
291381,2000-12-31,86746,20103,032280,STARZ,54268.000
291389,2001-12-31,89130,41998,032280,STARZ,48539.000
291396,2002-12-31,89130,41998,032280,STARZ,39685.000
291403,2003-12-31,89130,41998,032280,STARZ,54013.000
291410,2004-12-31,89130,41998,032280,STARZ,50181.000


### Many GVKEYs for one PERMCO

In [21]:
df[df['at'].notnull()].groupby('permco').gvkey.nunique().max()

5

In [22]:
pp = df[df['at'].notnull()].groupby('permco').gvkey.nunique().idxmax()

In [23]:
df.loc[df.permco==pp, ['datadate', 'permno', 'permco','gvkey', 'conm', 'at']].sort_values(['datadate'])

,datadate,permno,permco,gvkey,conm,at
336696,1994-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,NaN
336697,1995-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,NaN
157632,1996-12-31,10324,8243,012233,GENZYME CORP,1229.519
336698,1996-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,42.593
157633,1997-12-31,10324,8243,012233,GENZYME CORP,1203.056
336699,1997-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,56.818
335801,1998-12-31,86426,8243,117298,GENZYME MOLECULAR ONCOLOGY,35.952
336700,1998-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,18.954
157634,1998-12-31,10324,8243,012233,GENZYME CORP,1646.307
336701,1999-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,19.648


### Many GVKEYs for one PERMCO for the Same Date

In [24]:
df[df['at'].notnull()].groupby(['permco','datadate']).gvkey.nunique().max()

4

In [26]:
pp = df[df['at'].notnull()].groupby(['permco','datadate']).gvkey.nunique().idxmax()
pp

(8243, Timestamp('1999-12-31 00:00:00'))

In [27]:
df.loc[df.permco==8243, ['datadate', 'permno', 'permco','gvkey', 'conm', 'at']].sort_values(['datadate'])

,datadate,permno,permco,gvkey,conm,at
336696,1994-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,NaN
336697,1995-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,NaN
157632,1996-12-31,10324,8243,012233,GENZYME CORP,1229.519
336698,1996-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,42.593
157633,1997-12-31,10324,8243,012233,GENZYME CORP,1203.056
336699,1997-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,56.818
335801,1998-12-31,86426,8243,117298,GENZYME MOLECULAR ONCOLOGY,35.952
336700,1998-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,18.954
157634,1998-12-31,10324,8243,012233,GENZYME CORP,1646.307
336701,1999-12-31,81154,8243,118653,GENZYME TISSUE REPAIR,19.648


In [28]:
df.loc[df.permco==54311, ['datadate', 'permno', 'permco','gvkey', 'conm', 'at']].sort_values(['datadate'])

,datadate,permno,permco,gvkey,conm,at
209884,2013-12-31,13757,54311,017010,LIBERTY MEDIA SIRIUSXM GROUP,34542.0
209885,2013-12-31,13758,54311,017010,LIBERTY MEDIA SIRIUSXM GROUP,34542.0
209892,2014-12-31,13757,54311,017010,LIBERTY MEDIA SIRIUSXM GROUP,31207.0
209893,2014-12-31,13758,54311,017010,LIBERTY MEDIA SIRIUSXM GROUP,31207.0
209894,2014-12-31,14811,54311,017010,LIBERTY MEDIA SIRIUSXM GROUP,31207.0
209900,2015-12-31,13757,54311,017010,LIBERTY MEDIA SIRIUSXM GROUP,29798.0
209901,2015-12-31,13758,54311,017010,LIBERTY MEDIA SIRIUSXM GROUP,29798.0
209902,2015-12-31,14811,54311,017010,LIBERTY MEDIA SIRIUSXM GROUP,29798.0
259702,2016-12-31,14811,54311,027187,LIBERTY MEDIA FORMULA ONE,2995.0
259700,2016-12-31,13757,54311,027187,LIBERTY MEDIA FORMULA ONE,2995.0
